<H1> Comparing London vs New York Neighborhoods: Diversity and Presence of Parks</H1>

<H2> Coursera Week 4 </H2>

<H3> 1. Load the libraries </H3>

In [88]:


import numpy as np # library to handle data in a vectorized manner
!pip install geopy
!pip install folium
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Pyth

<H3> 2. Pulling  and Preping London data </H3>

In [89]:

#create London empty data frame
# define the dataframe columns
london_column = ['Borough', 'Inner','Status', 'Local authority', 'Political control', 'Headquarters', 'Area', 'Population', 'Co-ordinates', 'Nr in map'] 

# instantiate the dataframe
london= pd.DataFrame(columns=london_column)

URL = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
london.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Co-ordinates,Nr in map


In [90]:
#Create London DF
for items in soup.find_all('tr')[1::1]:
    data = items.find_all('td')
    try:
        Bor=data[0].text
        Boro =Bor.replace("\n"," ") 
        Borou=Boro.replace("[note 1]"," ")
        Boroug=Borou.replace("[note 2]"," ")
        Borough=Boroug.replace("[note 4]"," ")
        Inn=data[1].text
        Inner=Inn.rstrip('\n')
        Stat=data[2].text
        Status=Stat.replace("\n"," ")
        Aut=data[3].text
        Auth=Aut.replace("\n"," ")
        Cont=data[4].text
        Control=Cont.replace("\n","")
        head=data[5].text
        headq=head.replace("\n","")
        ar=data[6].text
        area=ar.replace("\n","")
        po=data[7].text
        pop=po.replace("\n","")
        coor=data[8].text
        coord=coor.replace("\n","")
        ma=data[9].text
        map=ma.replace("\n","");
    except IndexError:pass
    london = london.append({'Borough': Borough,
                            'Inner': Inner,
                            'Status': Status,
                            'Local authority': Auth,
                            'Political control': Control,
                            'Headquarters': headq,
                            'Area': area,
                            'Population': pop,
                            'Co-ordinates': coord,
                            'Nr in map': map}, ignore_index=True)


#check the table
london.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Co-ordinates,Nr in map
0,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
1,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20


In [94]:
london=london[:32]
london

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Co-ordinates,Nr in map,Latitude,Longitude
0,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25,51.5607,0.1557
1,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31,51.6252,-0.1517
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23,51.4549,0.1505
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12,51.5588,-0.2817
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20,51.4039,0.0198
5,Camden,Y,,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,11,51.5290,-0.1255
6,Croydon,,,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,19,51.3714,-0.0977
7,Ealing,,,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,13,51.5130,-0.3089
8,Enfield,,,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,30,51.6538,-0.0799
9,Greenwich,Y [note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,"264,008",51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...,22,51.4892,0.0648


In [91]:
#extracting longitude and latitude data
london['Latitude']=london['Co-ordinates'].str.split(r'\s*/\s*|\s*\°\s*|\s*\N\s*').str[4]
london['Longitude']=london['Co-ordinates'].str.split(r'\s*/\s*|\s*\°\s*|\s*\N\s|\s*\ \s*\*').str[8]
london['Longitude']=london['Longitude'].str.split(r'\s*\;\s*|\s*\ \s*').str[1]
london.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Co-ordinates,Nr in map,Latitude,Longitude
0,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25,﻿51.5607,0.1557﻿
1,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31,﻿51.6252,-0.1517﻿
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23,﻿51.4549,0.1505﻿
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12,﻿51.5588,-0.2817﻿
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20,﻿51.4039,0.0198﻿


In [92]:
#format latitude and longitude into floats
london['Longitude'] = london['Longitude'] .map(lambda x: x.lstrip('\ufeff').rstrip('\ufeff'))
london['Latitude'] = london['Latitude'] .map(lambda x: x.lstrip('\ufeff').rstrip('\ufeff'))
london['Longitude']=london['Longitude'].astype(float)
london['Latitude']=london['Latitude'].astype(float)
london.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Co-ordinates,Nr in map,Latitude,Longitude
0,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25,51.5607,0.1557
1,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31,51.6252,-0.1517
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23,51.4549,0.1505
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12,51.5588,-0.2817
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20,51.4039,0.0198


In [93]:
london

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Co-ordinates,Nr in map,Latitude,Longitude
0,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25,51.5607,0.1557
1,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31,51.6252,-0.1517
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23,51.4549,0.1505
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12,51.5588,-0.2817
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20,51.4039,0.0198
5,Camden,Y,,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,11,51.5290,-0.1255
6,Croydon,,,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,19,51.3714,-0.0977
7,Ealing,,,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,13,51.5130,-0.3089
8,Enfield,,,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,30,51.6538,-0.0799
9,Greenwich,Y [note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,"264,008",51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...,22,51.4892,0.0648


<H3> 3. Pulling NY data </H3>

In [95]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']


Data downloaded!


In [96]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [97]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<H3> 4. Map Both London and NY neighborhoods </H3>

In [98]:
#4 Map both data
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [99]:
# create map of New York using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(london['Latitude'], london['Longitude'], london['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7
        #parse_html=False
    ).add_to(map_london)  
    
map_london


In [100]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [101]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
        #parse_html=False
    ).add_to(map_newyork)  
    
map_newyork

<H3> 5. Add Foursquare top 5 per Neighborhood data  </H3>

In [102]:
# The code was removed by Watson Studio for sharing.

<H3> 5.1 London venue data </H3>

In [105]:
#create London Venue dataframe
neighborhood_latitude = london['Latitude'] # neighborhood latitude value
neighborhood_longitude = london['Longitude'] # neighborhood longitude value

neighborhood_name = london['Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()

Latitude and longitude values of 0       Barking and Dagenham   
1                       Barnet 
2                       Bexley 
3                        Brent 
4                      Bromley 
5                       Camden 
6                      Croydon 
7                       Ealing 
8                      Enfield 
9                  Greenwich   
10                     Hackney 
11    Hammersmith and Fulham   
12                    Haringey 
13                      Harrow 
14                    Havering 
15                  Hillingdon 
16                    Hounslow 
17                   Islington 
18      Kensington and Chelsea 
19        Kingston upon Thames 
20                     Lambeth 
21                    Lewisham 
22                      Merton 
23                      Newham 
24                   Redbridge 
25        Richmond upon Thames 
26                   Southwark 
27                      Sutton 
28               Tower Hamlets 
29              Waltham Forest 
30     

In [106]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [115]:
#creating list of London venues
london_venues = getNearbyVenues(names=london['Borough'],
                                   latitudes=london['Latitude'],
                                   longitudes=london['Longitude']
                                  )

#check the size of the df
#print(london_venues.shape)
#london_venues.head()
#number of venues
london_venues.groupby('Neighborhood').count()
#Analyse Neighborhood
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 
#london_onehot.head()
# move neighborhood column to the first column
cols = london_onehot.columns.tolist()
n = int(cols.index('Neighborhood'))
cols = [cols[n]] + cols[:n] + cols[n+1:]
london_onehot = london_onehot[cols]

london_onehot.head()
#group beighborhoods
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped.head()

#get top 10 venues per neighbourhoods
num_top_venues = 10

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
#sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
#creating a new dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
london_venues_sorted = pd.DataFrame(columns=columns)
london_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    london_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

london_venues_sorted.head()


Barking and Dagenham   
Barnet 
Bexley 
Brent 
Bromley 
Camden 
Croydon 
Ealing 
Enfield 
Greenwich   
Hackney 
Hammersmith and Fulham   
Haringey 
Harrow 
Havering 
Hillingdon 
Hounslow 
Islington 
Kensington and Chelsea 
Kingston upon Thames 
Lambeth 
Lewisham 
Merton 
Newham 
Redbridge 
Richmond upon Thames 
Southwark 
Sutton 
Tower Hamlets 
Waltham Forest 
Wandsworth 
Westminster 
----Barking and Dagenham   ----
                  venue  freq
0           Bus Station  0.14
1                  Pool  0.14
2           Golf Course  0.14
3           Supermarket  0.14
4  Gym / Fitness Center  0.14
5     Martial Arts Dojo  0.14
6                  Park  0.14
7      Pedestrian Plaza  0.00
8          Optical Shop  0.00
9       Organic Grocery  0.00


----Barnet ----
                        venue  freq
0                        Café  0.50
1                    Bus Stop  0.25
2  Construction & Landscaping  0.25
3          African Restaurant  0.00
4         Peruvian Restaurant  0.00
5           Outd

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Pool,Gym / Fitness Center,Bus Station,Supermarket,Golf Course,Martial Arts Dojo,Park,French Restaurant,Food Court,Food
1,Barnet,Café,Bus Stop,Construction & Landscaping,Yoga Studio,Film Studio,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
2,Bexley,Pub,Italian Restaurant,Coffee Shop,Clothing Store,Fast Food Restaurant,Supermarket,Bakery,Pharmacy,Portuguese Restaurant,Sandwich Place
3,Brent,Bar,Hotel,Coffee Shop,Sporting Goods Shop,Clothing Store,Sandwich Place,Indian Restaurant,Restaurant,Italian Restaurant,American Restaurant
4,Bromley,Coffee Shop,Clothing Store,Burger Joint,Gym / Fitness Center,Pizza Place,Bar,Sandwich Place,Chocolate Shop,Pub,Stationery Store


In [120]:
neighborhoods.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

<H3> 5.2. NY Venue data </H3>

In [127]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [128]:
# type your answer here
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [129]:
print(ny_venues.shape)
ny_venues.head()

(10242, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop
4,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place


In [130]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,32,32,32,32,32,32
Annadale,9,9,9,9,9,9
Arden Heights,5,5,5,5,5,5
Arlington,8,8,8,8,8,8
Arrochar,20,20,20,20,20,20
Arverne,15,15,15,15,15,15
Astoria,100,100,100,100,100,100
Astoria Heights,14,14,14,14,14,14
Auburndale,21,21,21,21,21,21


In [132]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 


,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North India

In [133]:
# move neighborhood column to the first column
cols = ny_onehot.columns.tolist()
n = int(cols.index('Neighborhood'))
cols = [cols[n]] + cols[:n] + cols[n+1:]
ny_onehot = ny_onehot[cols]
ny_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North India

In [134]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()

----Allerton----
                venue  freq
0         Pizza Place  0.16
1                 Spa  0.09
2         Supermarket  0.09
3       Deli / Bodega  0.06
4  Chinese Restaurant  0.06
5      Discount Store  0.03
6          Donut Shop  0.03
7      Cosmetics Shop  0.03
8   Electronics Store  0.03
9          Playground  0.03


----Annadale----
                   venue  freq
0          Train Station  0.11
1         Cosmetics Shop  0.11
2              Pet Store  0.11
3                    Pub  0.11
4                  Diner  0.11
5             Sports Bar  0.11
6            Pizza Place  0.11
7           Liquor Store  0.11
8             Restaurant  0.11
9  Performing Arts Venue  0.00


----Arden Heights----
                  venue  freq
0              Pharmacy   0.2
1         Deli / Bodega   0.2
2           Pizza Place   0.2
3          Home Service   0.2
4           Coffee Shop   0.2
5  Pakistani Restaurant   0.0
6             Pet Store   0.0
7           Pet Service   0.0
8              Pet Ca

In [136]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_venues_sorted = pd.DataFrame(columns=columns)
ny_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    ny_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Spa,Supermarket,Deli / Bodega,Chinese Restaurant,Spanish Restaurant,Food,Fast Food Restaurant,Pharmacy,Martial Arts Dojo
1,Annadale,Pet Store,Pub,Liquor Store,Train Station,Pizza Place,Sports Bar,Diner,Cosmetics Shop,Restaurant,Ethiopian Restaurant
2,Arden Heights,Pharmacy,Deli / Bodega,Pizza Place,Coffee Shop,Home Service,Field,Event Service,Event Space,Exhibit,Eye Doctor
3,Arlington,Bus Stop,Boat or Ferry,Intersection,Deli / Bodega,Coffee Shop,American Restaurant,Food Service,Women's Store,Filipino Restaurant,Field
4,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Liquor Store,Pizza Place,Supermarket,Taco Place,Middle Eastern Restaurant,Mediterranean Restaurant,Bagel Shop


<H3> 6. Clustering and optimising the K </H3>

<H4> 6.1 Clustering London Neighborhoods </H4>

In [137]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
london_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
london_kmeans.labels_[0:10] 

array([0, 2, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [156]:
#optimising the cluster groups for London and running the Kmeans clustering again with optimal Kclusters
from sklearn.cluster import KMeans
 
import matplotlib.pyplot as plt
%matplotlib inline

for k in range (1, 15):
 
    # Create a kmeans model on our data, using k clusters.  random_state helps ensure that the algorithm returns the same results each time.
    kmeans_model = KMeans(n_clusters=k, random_state=1).fit(london_grouped_clustering)

    # These are our fitted labels for clusters -- the first cluster has label 0, and the second has label 1.
    labels = kmeans_model.labels_
 
   # Sum of distances of samples to their closest cluster center
    inertia = kmeans_model.inertia_
    print ("k:",k, " cost:", inertia)


k: 1  cost: 1.6524737685
k: 2  cost: 1.3067030158
k: 3  cost: 1.12915244906
k: 4  cost: 0.979315211293
k: 5  cost: 0.837692382891
k: 6  cost: 0.726264642706
k: 7  cost: 0.61724012825
k: 8  cost: 0.549523122294
k: 9  cost: 0.493905279128
k: 10  cost: 0.45279461837
k: 11  cost: 0.405974000262
k: 12  cost: 0.370304244478
k: 13  cost: 0.338292749953
k: 14  cost: 0.312049438545


In [174]:
#selecting 7 clusters
kclusters = 10

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
london_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
london_kmeans.labels_[0:10] 

array([4, 3, 1, 0, 1, 0, 0, 1, 1, 1], dtype=int32)

In [175]:
# create map
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

london['Neighborhood']=london['Borough']
london_merged = london

# add clustering labels
london_merged['Cluster Labels'] = london_kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(london_venues_sorted.set_index('Neighborhood'), on='Borough')

london_merged.head() # check the last columns!

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(1,kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<H4> 6.2 Clustering and optimising NY data </H4>

In [159]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
ny_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
ny_kmeans.labels_[0:10] 

array([0, 0, 3, 3, 3, 0, 0, 0, 0, 0], dtype=int32)

In [161]:
#optimising the cluster groups for London and running the Kmeans clustering again with optimal Kclusters
for k in range (1, 25):
 
    # Create a kmeans model on our data, using k clusters.  random_state helps ensure that the algorithm returns the same results each time.
    kmeans_model = KMeans(n_clusters=k, random_state=1).fit(ny_grouped_clustering)

    # These are our fitted labels for clusters -- the first cluster has label 0, and the second has label 1.
    labels = kmeans_model.labels_
 
   # Sum of distances of samples to their closest cluster center
    inertia = kmeans_model.inertia_
    print ("k:",k, " cost:", inertia)


k: 1  cost: 28.2613251173
k: 2  cost: 26.7353885853
k: 3  cost: 25.6294965723
k: 4  cost: 24.560677156
k: 5  cost: 24.0011331907
k: 6  cost: 22.9633727944
k: 7  cost: 22.4946489889
k: 8  cost: 22.1298645637
k: 9  cost: 21.3053427935
k: 10  cost: 20.6973094895
k: 11  cost: 19.7502578274
k: 12  cost: 19.6466947839
k: 13  cost: 18.7299240503
k: 14  cost: 18.5718083924
k: 15  cost: 18.2765509087
k: 16  cost: 17.691207953
k: 17  cost: 18.0681051909
k: 18  cost: 17.3965117801
k: 19  cost: 17.1446157119
k: 20  cost: 17.2871776692
k: 21  cost: 16.8595873246
k: 22  cost: 16.5680085912
k: 23  cost: 16.2461398553
k: 24  cost: 16.1167479657


In [221]:
# set number of clusters
kclusters = 17

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
ny_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
ny_kmeans.labels_[0:10] 

array([ 1,  9, 15,  8,  2, 14, 12,  1,  9,  9], dtype=int32)

In [223]:
ny_grouped['Cluster Labels']=ny_kmeans.labels_

In [198]:
from collections import Counter
Counter(ny_kmeans.labels_)

Counter({0: 1,
         1: 30,
         2: 19,
         3: 1,
         4: 8,
         5: 1,
         6: 1,
         7: 20,
         8: 9,
         9: 81,
         10: 1,
         11: 1,
         12: 85,
         13: 1,
         14: 18,
         15: 21,
         16: 3})

In [199]:
# create map
address = 'New York, United States'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged=pd.merge(ny_venues_sorted, neighborhoods , left_on = 'Neighborhood', right_on = 'Neighborhood', how = 'left')

In [231]:
ny_merged=ny_grouped[['Neighborhood', 'Cluster Labels']]
ny_merged1=pd.merge(ny_merged, neighborhoods , left_on = 'Neighborhood', right_on = 'Neighborhood', how = 'left')
ny_merged1 =ny_merged1.join(ny_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# create map
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged1['Latitude'], ny_merged1['Longitude'], ny_merged1['Neighborhood'], ny_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


<H3> 7 Examining the clusters </H3>

In [238]:
london_merged.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area,Population,Co-ordinates,Nr in map,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25,51.5607,0.1557,4,Barking and Dagenham,Pool,Gym / Fitness Center,Bus Station,Supermarket,Golf Course,Martial Arts Dojo,Park,French Restaurant,Food Court,Food
1,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31,51.6252,-0.1517,3,Barnet,Café,Bus Stop,Construction & Landscaping,Yoga Studio,Film Studio,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23,51.4549,0.1505,1,Bexley,Pub,Italian Restaurant,Coffee Shop,Clothing Store,Fast Food Restaurant,Supermarket,Bakery,Pharmacy,Portuguese Restaurant,Sandwich Place
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12,51.5588,-0.2817,0,Brent,Bar,Hotel,Coffee Shop,Sporting Goods Shop,Clothing Store,Sandwich Place,Indian Restaurant,Restaurant,Italian Restaurant,American Restaurant
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20,51.4039,0.0198,1,Bromley,Coffee Shop,Clothing Store,Burger Joint,Gym / Fitness Center,Pizza Place,Bar,Sandwich Place,Chocolate Shop,Pub,Stationery Store


In [244]:
#create summary table for london
london_col = ['1st common', '2nd Common','3rd Common', '4th Common', '5th Common', 'Total'] 

# instantiate the dataframe
london_summ= pd.DataFrame(columns=london_col)
london_summ['1st common']=london_merged['1st Most Common Venue'].value_counts()
london_summ['2nd Common']=london_merged['2nd Most Common Venue'].value_counts()
london_summ['3rd Common']=london_merged['3rd Most Common Venue'].value_counts()
london_summ['4th Common']=london_merged['4th Most Common Venue'].value_counts()
london_summ['5th Common']=london_merged['5th Most Common Venue'].value_counts()
london_summ.fillna(0)


,1st common,2nd Common,3rd Common,4th Common,5th Common,Total
Coffee Shop,9,2.0,7.0,1.0,1.0,0
Pub,7,6.0,3.0,2.0,2.0,0
Clothing Store,3,6.0,0.0,2.0,2.0,0
Park,2,0.0,0.0,0.0,0.0,0
Café,2,2.0,2.0,1.0,4.0,0
Caribbean Restaurant,1,0.0,0.0,0.0,0.0,0
Fast Food Restaurant,1,2.0,0.0,0.0,2.0,0
Sandwich Place,1,0.0,0.0,2.0,0.0,0
Hotel,1,5.0,1.0,3.0,0.0,0
Bar,1,0.0,0.0,1.0,1.0,0


In [252]:
col_list=['1st common','2nd Common','3rd Common','4th Common','5th Common']
london_summ['Total'] = london_summ[col_list].sum(axis=1)
london_summ['Percent']=london_summ['Total']/(london_summ['Total'].sum())*100
london_summ.fillna(0)

,1st common,2nd Common,3rd Common,4th Common,5th Common,Total,Percent
Coffee Shop,9,2.0,7.0,1.0,1.0,20.0,19.230769
Pub,7,6.0,3.0,2.0,2.0,20.0,19.230769
Clothing Store,3,6.0,0.0,2.0,2.0,13.0,12.500000
Park,2,0.0,0.0,0.0,0.0,2.0,1.923077
Café,2,2.0,2.0,1.0,4.0,11.0,10.576923
Caribbean Restaurant,1,0.0,0.0,0.0,0.0,1.0,0.961538
Fast Food Restaurant,1,2.0,0.0,0.0,2.0,5.0,4.807692
Sandwich Place,1,0.0,0.0,2.0,0.0,3.0,2.884615
Hotel,1,5.0,1.0,3.0,0.0,10.0,9.615385
Bar,1,0.0,0.0,1.0,1.0,3.0,2.884615


In [254]:
#create summary table for NY
ny_col = ['1st common', '2nd Common','3rd Common', '4th Common', '5th Common', 'Total'] 

# instantiate the dataframe
ny_summ= pd.DataFrame(columns=london_col)
ny_summ['1st common']=ny_merged1['1st Most Common Venue'].value_counts()
ny_summ['2nd Common']=ny_merged1['2nd Most Common Venue'].value_counts()
ny_summ['3rd Common']=ny_merged1['3rd Most Common Venue'].value_counts()
ny_summ['4th Common']=ny_merged1['4th Most Common Venue'].value_counts()
ny_summ['5th Common']=ny_merged1['5th Most Common Venue'].value_counts()
ny_summ.fillna(0)


,1st common,2nd Common,3rd Common,4th Common,5th Common,Total
Italian Restaurant,41,17.0,11.0,5.0,5.0,0
Pizza Place,31,21.0,16.0,22.0,17.0,0
Deli / Bodega,18,24.0,14.0,8.0,6.0,0
Chinese Restaurant,16,6.0,6.0,7.0,6.0,0
Coffee Shop,15,12.0,12.0,5.0,5.0,0
Bar,13,9.0,7.0,10.0,3.0,0
Caribbean Restaurant,11,2.0,3.0,2.0,2.0,0
Bus Stop,8,3.0,4.0,5.0,4.0,0
Beach,8,1.0,3.0,0.0,0.0,0
Park,8,11.0,6.0,9.0,3.0,0


In [256]:
col_list=['1st common','2nd Common','3rd Common','4th Common','5th Common']
ny_summ['Total'] = ny_summ[col_list].sum(axis=1)
ny_summ['Percent']=ny_summ['Total']/(ny_summ['Total'].sum())*100
ny_summ.fillna(0)

,1st common,2nd Common,3rd Common,4th Common,5th Common,Total,Percent
Italian Restaurant,41,17.0,11.0,5.0,5.0,79.0,6.661046
Pizza Place,31,21.0,16.0,22.0,17.0,107.0,9.021922
Deli / Bodega,18,24.0,14.0,8.0,6.0,70.0,5.902192
Chinese Restaurant,16,6.0,6.0,7.0,6.0,41.0,3.456998
Coffee Shop,15,12.0,12.0,5.0,5.0,49.0,4.131535
Bar,13,9.0,7.0,10.0,3.0,42.0,3.541315
Caribbean Restaurant,11,2.0,3.0,2.0,2.0,20.0,1.686341
Bus Stop,8,3.0,4.0,5.0,4.0,24.0,2.023609
Beach,8,1.0,3.0,0.0,0.0,12.0,1.011804
Park,8,11.0,6.0,9.0,3.0,37.0,3.119730
